In [1]:
import json
import onnx
import torch
import numpy as np
import pandas as pd
from torch import nn

In [2]:
class ExampleModel(nn.Module):

    def __init__(self):
        super(ExampleModel, self).__init__()

    def forward(self, x1, x2, x3):
        # Input x1 is a scalar
        # Input x2 is a vector with causality 'local' and 4 elements
        # Input x3 is a vector with causality 'countinuous' and 5 elements
        x4 = x2 + x3[:4]
        x5 = x2 - x3[:4]
        x6 = x1 * x3[-1]
        x7 = x1 / x3[-1]
        x = torch.cat([x4, x5, x6, x7])
        return x

In [3]:
# Create three tensors
x1 = torch.tensor(1.0).unsqueeze(-1)
x2 = torch.randn(4)
x3 = torch.randn(5)

# Print tensors shape
print(x1.shape)
print(x2.shape)
print(x3.shape)

# Create the model
model = ExampleModel()

# Run the model
output = model(x1, x2, x3)
print(output)


torch.Size([1])
torch.Size([4])
torch.Size([5])
tensor([ 1.0795e+00, -3.8908e-01, -5.9069e-01,  3.7054e-01, -1.5823e+00,
        -4.8325e-01, -3.1670e-03,  1.1996e+00, -3.0126e-01, -3.3194e+00])


In [4]:
# Set the model to evaluation mode
model.eval()

# Save the model in ONNX format
torch.onnx.export(
    model,
    (x1, x2, x3),
    "example1.onnx",
    verbose=True,
    input_names=["scalar_input", "vector_input", "vector_input_discrete"],
    output_names=["output"],
)

# Load the model
model_name = "example1"
onnx_model = onnx.load(f"{model_name}.onnx")

# Check the model
onnx.checker.check_model(onnx_model)

# Add description to the model
onnx_model.graph.doc_string = "Example model with scalar and vector inputs"

# Add descrption to the input
onnx_model.graph.input[0].doc_string = "Scalar input"
onnx_model.graph.input[1].doc_string = "Vector input with causality local"
onnx_model.graph.input[2].doc_string = "Vector input with causality continuous"
onnx_model.graph.output[0].doc_string = "Output"

# Add metadata to the model
onnx_model.producer_name = "ExampleModel"
onnx_model.producer_version = "0.0.1"
onnx_model.domain = "example"
onnx_model.model_version = 1

# Save the model
onnx.save(onnx_model, f"{model_name}.onnx")


Exported graph: graph(%scalar_input : Float(1, strides=[1], requires_grad=0, device=cpu),
      %vector_input : Float(4, strides=[1], requires_grad=0, device=cpu),
      %vector_input_discrete : Float(5, strides=[1], requires_grad=0, device=cpu)):
  %/Constant_output_0 : Long(1, strides=[1], requires_grad=0, device=cpu) = onnx::Constant[value={0}, onnx_name="/Constant"](), scope: __main__.ExampleModel:: # /tmp/ipykernel_20273/3488084788.py:10:0
  %/Constant_1_output_0 : Long(1, strides=[1], requires_grad=0, device=cpu) = onnx::Constant[value={0}, onnx_name="/Constant_1"](), scope: __main__.ExampleModel:: # /tmp/ipykernel_20273/3488084788.py:10:0
  %/Constant_2_output_0 : Long(1, strides=[1], requires_grad=0, device=cpu) = onnx::Constant[value={4}, onnx_name="/Constant_2"](), scope: __main__.ExampleModel:: # /tmp/ipykernel_20273/3488084788.py:10:0
  %/Constant_3_output_0 : Long(1, strides=[1], requires_grad=0, device=cpu) = onnx::Constant[value={1}, onnx_name="/Constant_3"](), scope: __

## Generating model description

Create and save the model description to be provided to ONNX2FMU.

In [5]:
model_description = {
    "name": "example1",
    "description": "The model defines a simple example model with a scalar input and two vector inputs, one with 'local' variability and one with 'continuous' variability.",
    "FMIVersion": "2.0",
    "inputs": [
        {
            "name": "scalar_input",
            "description": "A scalar input to the model."
        },
        {
            "name": "vector_input",
            "description": "A vector of input variables with variability discrete."
        },
        {
            "name": "vector_input_discrete",
            "description": "Inputs have variability discrete by default."
        }
    ],
    "outputs": [
        {
            "name": "output",
            "description": "The output array.",
            "labels": ["Class1", "Class2", "Class3", "Class4", "Class5", "Class6", "Class7", "Class8", "Class9", "Class10"]
        }
    ]
}

# Save model description
with open(f"{model_name}Description.json", "w", encoding="utf-8") as f:
    json.dump(model_description, f, indent=4)

## Generating input file and output for testing

In [7]:
time_steps = 100
# Create 10 arrays of random numbers
data = np.random.random((time_steps, 10))
scalar_input = data[:, 0]
vector_input_local = data[:, 1:5]
vector_input_continuous = data[:, 5:]

# Do calculations like in the model
out1 = vector_input_local + vector_input_continuous[:, :4]
out2 = vector_input_local - vector_input_continuous[:, :4]
out3 = scalar_input * vector_input_continuous[:, -1]
out4 = scalar_input / vector_input_continuous[:, -1]

out = np.hstack([out1, out2, out3[:, None], out4[:, None]])

# Compare numpy output with model output
scalar_input_tensor = torch.tensor(scalar_input[:, None])
vector_input_local_tensor = torch.tensor(vector_input_local)
vector_input_continuous_tensor = torch.tensor(vector_input_continuous)

y = []

for i in range(time_steps):
    y.append(
        model(
            scalar_input_tensor[i],
            vector_input_local_tensor[i],
            vector_input_continuous_tensor[i]
        )
    )

y = torch.stack(y).numpy()

mse = np.sum(np.pow(y - out, 2))

print(f"Mean squared error {mse}")

# Generate time column
time = np.arange(time_steps)

# Initialize input dataset
columns_data = []
for i, node in enumerate(onnx_model.graph.input):
    description = model_description['inputs'][i]
    shape = tuple(
        dim.dim_value for dim in node.type.tensor_type.shape.dim
    )
    # If tensor shape is empty, set it to 1
    if not shape:
        shape = (1,)
    # Define array names
    columns_data += [
        description['name'] + "_" + "_".join([str(k) for k in idx])
        for idx in np.ndindex(shape)
    ]
df_data = pd.DataFrame(data, columns=columns_data)
df_data['time'] = time
df_data.set_index('time', inplace=True)
df_data.to_csv('input.csv')

# Initialize out
df_y = pd.DataFrame(y, columns=[f"output_{i}" for i in range(y.shape[1])])
df_y['time'] = time
df_y.set_index('time', inplace=True)
df_y.to_csv('output.csv')


Mean squared error 0.0
